In [ ]:
# TESTİNG: Kaydedilmiş modeller yüklenip test edildi ve sonuçları raporlandı bu bölümde.

import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import os
import warnings
warnings.filterwarnings('ignore')

ULKELER = ['tr', 'fi', 'au'] 
MODELLER = ['xgboost', 'randomforest', 'mlp', 'knn']
POLLUTANTS = ['co', 'no2', 'o3', 'pm10', 'pm25', 'so2']

all_sonucs = []
for country_code in ULKELER:
    print(f"\n\nÜLKE: {country_code.upper()} İÇİN TESTLER BAŞLIYOR")
    
    for model_name in MODELLER:
        for target in POLLUTANTS:
            model_path = f'model_{model_name}_{target}_{country_code}.pkl'
            if not os.path.exists(model_path):
                continue # Model yoksa bu kombinasyonu atladı

            print(f"\n Test Ediliyor: Model={model_name.upper()}, Kirletici={target.upper()}")
            
            try:
                # Gerekli tüm nesnelerin yüklenmesi
                model = joblib.load(model_path)
                x_scaler = joblib.load(f'x_scaler_{model_name}_{target}_{country_code}.pkl')
                y_scaler = joblib.load(f'y_scaler_{model_name}_{target}_{country_code}.pkl')
                selector = joblib.load(f'selector_{model_name}_{target}_{country_code}.pkl')
                X_test = pd.read_csv(f'X_test_{target}_{country_code}.csv', index_col='from_date', parse_dates=True)
                y_test = pd.read_csv(f'y_test_{target}_{country_code}.csv', index_col='from_date', parse_dates=True)
                
                # Test verisini dönüşümü
                X_test_s = x_scaler.transform(X_test)
                X_test_f = selector.transform(X_test_s)
                
                # Tahmin yapılır ve sonuçları orijinal ölçeğe geri çevrilir
                tahmin_s = model.predict(X_test_f)
                tahmin_log = y_scaler.inverse_transform(tahmin_s.reshape(-1, 1))
                tahmin_final = np.expm1(tahmin_log).flatten()
                y_test_final = np.expm1(y_test.values.flatten())
                
                # Metrikleri Hesaplanır ve Sonuçları Kaydedilir
                rmse = np.sqrt(mean_squared_error(y_test_final, tahmin_final))
                r2 = r2_score(y_test_final, tahmin_final)
                mae = mean_absolute_error(y_test_final, tahmin_final)
                
                print(f"  RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
                
                sonuc = {'Ülke': country_code.upper(), 'Model': model_name, 'Kirletici': target, 'RMSE': rmse, 'R2': r2, 'MAE': mae}
                all_sonucs.append(sonuc)

            except FileNotFoundError as e:
                print(f"Hata: Dosya bulunamadı ({e.filename})")
                continue

# özet tablosu
if all_sonucs:
    df_summary = pd.DataFrame(all_sonucs)
    print("\n\n" + "="*30 + " TÜM MODELLERİN ÖZET TABLOSU " + "="*30)
    
    pivot_r2 = df_summary.pivot_table(index='Kirletici', columns=['Ülke', 'Model'], values='R2')
    print("\n--- R-Squared (R²) Skorları ---")
    print(pivot_r2.round(4))
    
    summary_filename = f'final_sonucs_summary.csv'
    df_summary.to_csv(summary_filename, index=False)
    print(f"\nDetaylı özet tablosu '{summary_filename}' dosyasına kaydedildi")



ÜLKE: TR İÇİN TESTLER BAŞLIYOR

 Test Ediliyor: Model=XGBOOST, Kirletici=CO
  RMSE: 23.5681, MAE: 11.7869, R²: 0.0000

 Test Ediliyor: Model=XGBOOST, Kirletici=NO2
  RMSE: 23.0616, MAE: 18.6303, R²: 0.3381

 Test Ediliyor: Model=XGBOOST, Kirletici=O3
  RMSE: 15.5161, MAE: 8.6085, R²: -1.4294

 Test Ediliyor: Model=XGBOOST, Kirletici=PM10
  RMSE: 188.6431, MAE: 36.4484, R²: -193.3308

 Test Ediliyor: Model=XGBOOST, Kirletici=PM25
  RMSE: 15.0213, MAE: 10.8668, R²: -0.6624

 Test Ediliyor: Model=XGBOOST, Kirletici=SO2
  RMSE: 0.3698, MAE: 0.1483, R²: 0.0000

 Test Ediliyor: Model=RANDOMFOREST, Kirletici=CO
  RMSE: 0.0031, MAE: 0.0031, R²: 0.0000

 Test Ediliyor: Model=RANDOMFOREST, Kirletici=NO2
  RMSE: 20.5658, MAE: 16.4390, R²: 0.4736

 Test Ediliyor: Model=RANDOMFOREST, Kirletici=O3
  RMSE: 11.4855, MAE: 8.1499, R²: -0.3312

 Test Ediliyor: Model=RANDOMFOREST, Kirletici=PM10
  RMSE: 115.2720, MAE: 57.9866, R²: -71.5618

 Test Ediliyor: Model=RANDOMFOREST, Kirletici=PM25
  RMSE: 11.9